# **Step 3 - Let's use a Tool**

Let's now brainstorm a possible solution to [Step 2's challenge](02_get_schedule.ipynb).  

Due to the potentially large number of VIP customers involved (potentially thousands), the idea of including all the special users IDs in the prompt is clearly not feasible.  

<!-- An alternative could be to ask GPT to analyze the presence of an ID in the prompt and, if found, return it, then identify somehow if that ID belongs to a VIP customer and return (with or without the help of the LLM) the proper schedule. 

This could potentially work, but imagine the user asking: *What is the schedule today for 730* (meaning 7:30 PM) would the LLM consider 730 as the ID?, or will correctly return "no id found" and what if in our solution we will have to handle with several different cases like this, like when the user enters it's id, the number of coke he wishes, the time in form *"in 20 minutes*"   

And how would we discriminate a request for the daily schedule for user 8643 from a request to cancel an order 8643? -->

### **Change the mindset**

Until now, we have tried to find a solution by using the LLM as tool to help us get what we need from the user (user's ID in this case).  

What if, instead, we instruct the LLM how to complete a specific task (e.g. Handle requests for the operating schedule) and give it the opportunity to "ask" for additional details in case it thinks that the amount of available information is not enough to complete it?  

Sounds complicated? 😲

As an example, imagine giving the following *pseudo*-instructions to the LLM.    

- You are an assistant in charge of handling requests about the opening schedule of a restaurant.
- Your task is to reply to requests coming from a user.
- In case you need to know if the restaurant is open or closed on a specific date, respond with "need schedule <the-date-you-need-to-check>"
- In this case I will fetch the schedule for that date and I will give it back to you.
- Then continue elaborating the request.
- Feel free to ask for schedules as many times you think is necessary.

Do you see the difference?   

We are now asking the LLM to **reason** on a given task and, if it deems necessary, **tell us** to **provide** it more information based on certain criteria; so that once that information is provided to it, the LLM can continue reasoning and complete the assigned task.

Ok, but *how can this be done technically?*

Let's see it in action.  We start with our usual AzureOpenAI client & data provider. 

In [ ]:
# Imports
import os
import json
import datetime
from openai import AzureOpenAI
from dotenv import load_dotenv
from datetime import datetime
from services.data_provider import DataProvider
from termcolor import cprint

# Environment setup
load_dotenv()
api_key=os.environ['AZURE_OPENAI_API_KEY']
deployment=os.environ['AZURE_OPENAI_DEPLOYMENT']
api_version=os.environ['AZURE_OPENAI_API_VERSION']

#Initialize AzureOpenAI client
client = AzureOpenAI(
  api_key=api_key,  
  api_version = api_version
  )

# This the data access layer
db:DataProvider=DataProvider()

messages = []

## Structure the system message 

Have now a look at the `system message`, as you see is now more structured and divided into sections that clearly describe the task that must be solved. 

However, we still have not introduced the concept that the LLM can ask us for additional information to help it accomplish the task. 

In [ ]:
# System message (We use RISEN syntax https://easyaibeginner.com/risen-framework-ai-prompt-for-chatgpt/)
system_message = f'''
Role: An assistant with expertise in handling questions about operative schedule of a food delivery service.
 
Instructions:
- Provide information about the opening schedule.
- Kindly deny any request not regarding opening schedule.
- Always assume the date and time provided as input the reference date for all date calculations.
- If a date is not indicated use the reference date.
- If a time is not indicated assume it is 12 PM.
- Assume the following format: 'DD/MM/YYYY HH:MM' as the standard format for dates.

Steps:
  - Opening Schedule:
      1. Always use the calculated date based on reference date to check and return the opening schedule.
      2. Always indicate the desired data and time including weekday in the response.
  
Expectation:
  - Provide a seamless experience to the user, by providing the requested information in a kind and timely manner, including all the necessary details and guiding the user to a possible follow up step.

Narrowing:  
  1. Deny all the requests referring to a date antecedent the reference date.
'''

## Introducing a function to look up the schedule for a particular user on a given day 

You will find below a function that will return the schedule for a given day in JSON format based a particular `date` and an **optional** `user id`.  For example, it will return the following schedule for Tuesday 25th June 2024:

```json
{
      "day": "Tuesday",
      "start": "9:00",
      "end": "21:00",
      "status": "open"
}
```
You don't need to understand the intricacies of the function.  What is important to remember is: what **goes in** and the corresponding **output**.

In [ ]:
def operative_schedule(date:str, user_id:str=None):
    """
    Retrieves and operating schedule for a given date and user.

    This function determines the operative schedule for a specified date. It first extracts the weekday from the provided date. 
    Then, it checks if the user is a special client and retrieves the schedule from the database accordingly. 
    If the schedule status is "open," the function verifies whether the requested date/time falls within the schedule's operational hours.
    If the time is outside the operational hours, it updates the schedule status to "closed."

    Parameters:
    ----------
    date : str
        The date and time in the format "dd/mm/YYYY HH:MM" for which to check the schedule.
    user_id : str, optional
        The unique identifier of the user. This is used to determine if the user is a special client. Default is None.

    Returns:
    -------
    str
        A JSON string representing the schedule model with updated status, if applicable.

    Notes:
    -----
    - The function relies on a database (`db`) with methods `is_special_client(user_id)` and `get_schedule(day, is_special)`.
    - The `get_schedule` method returns a schedule object with properties `status`, `start`, `end`, and `model_dump_json()`.
    - The function assumes the provided `date` string is in the correct format and valid.
    
    Example:
    -------
    >>> operative_schedule("23/06/2024 14:30", "1234")
    '{"status": "open", "start": "09:00", "end": "17:00"}'
    """
    weekday= datetime.strptime(date, "%d/%m/%Y %H:%M").strftime("%A")    
    
    # Check if the user is a special client
    is_special = db.is_special_client(user_id)
    # Get the operative schedule from the database
    schedule=db.get_schedule(day=weekday, is_special=is_special) 
    # Check if the requested date/time is within the working schedule, if not we update the status to closed.      
    if schedule.status == "open":        
        time= datetime.strptime(date, "%d/%m/%Y %H:%M").time()
        opening = datetime.strptime(schedule.start, "%H:%M").time()
        closing = datetime.strptime(schedule.end, "%H:%M").time()        
        if time < opening or time > closing:
            schedule.status = "closed"            
        
    return schedule.model_dump_json()

## Test the function

It's just a standard Python function. 

In [ ]:
operative_schedule('27/6/2024 00:00') 

## How can we inform the LLM that this function is available? 

As you might have noticed, in the system_message defined before there is no mention of the part: 
- *In case you think you need to know if the restaurant is open or closed on a specific date, respond with "need schedule <the-date-you-need-to-check>"*  


**How do we communicate this to the LLM?** 
 
For this we have to use a technique called [function calling](https://platform.openai.com/docs/guides/function-calling) that ChatGPT describes as:  

> The capability of the model to interact with external functions or APIs during a conversation or while processing a query.   
This enables the LLM to extend its capabilities beyond just generating text based on its training data, allowing it to perform specific tasks or retrieve real-time information through predefined functions

In short it means that we can tell the LLM to ask us (our code) to invoke a specific function with specified parameter values and then provide the result back to it.


## Ok, but *how do I do that?*

We do it by defining an object with a well defined structure like the one you see in the next cell.  (The format is an extended JSON schema)

Have a look and try to see what kind of information is present. 

In [ ]:
# Define the available functions to the LLM to use
tools = [
        {        
            "type": "function",    
            "function": {
                "name": "operative_schedule",
                "description": "Useful when you need to know the operative schedule on a specific date.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "date": {
                            "type": "string",
                            "description": "The date for which you want to know the schedule in format 'DD/MM/YYYY HH:MM' (e.g. '31/12/2023 15:00').",
                        },
                        "user_id": {
                            "type": "string",
                            "description": "The optional user id in form '#<4-digits-number>'.",
                        }                        
                    },
                    "required": ["date"]
                }
            },            
        }
    ]

What you see is that the JSON represents:
- An array of functions.
- Each function has:
  - A **Name** (*"operative_schedule"*)
  - A **Description** (*"Useful when you need to know the operative schedule on a specific date."*)
  - A **Variable number of parameters**, each one with its own **data type** and **descripion**
  - A list indicating which parameters are **mandatory** (*"date"*)

Note:
- The name of the function in the JSON matches the name of the function that we defined before. It's not a requirement, but makes the code easier to understand 😏
- The list of parameters also matches the ones we have in the preceding code.  

<br>

## Now, include the tool descriptions when invoking the LLM 

On it we have the same function you have already seen in the previous notebooks but with the following changes:

1. The input parameter is optional and when missing the LLM just uses what's on the messages list.
2. **Important**❗We now assign the `tools` object created before to the `tools` property of the `create` method.

So now our chat object is aware of:  

1. The **system message**
2. The **user message**
3. What **functions** it might request we call if necessary (via the `tools` object)

💡 While **function calling** is a general terms that applies to any language model, OpenAI uses the term **Tools**, so each time you read this term just think about a **code function** that has to be invoked.  

Now run the cell and continue...

In [ ]:
# Invokes the LLM with the provided input
def invoke_llm(input:str=None)->str:
    if input:
          messages.append({'role': 'user', 'content': input})
    response = client.chat.completions.create(
    model=deployment,    
    messages = messages,
    tools=tools,
    tool_choice="auto", # see https://platform.openai.com/docs/guides/function-calling/function-calling-behavior for more details,    
    temperature=0.7)

    return response.choices[0].message

## Recap 

Here's a representation of what we have build so far.  

<img src="../docs/images/agent-and-code.png" width="800"> 

As you see, there is something still unclear: *Who* is going to call the `operative_schedule` function, get the result and feed the result back into the LLM?

If you are thinking the LLM, the response is NO. The LLM is and remains stateless.  Instead the LLM informs you of the function and arguments to use, and you call the function. 

<br>

## Understanding the sequence 

What is going to happen is visible in the following sequence diagram  

<img src="../docs/images/tool-flow.png" width="800">

1. When the user submits a question, it gets forwarded to the LLM.
2. Based on the **system_message**, **tools** and **messages** content, the LLM will decide that it needs to 'invoke' the `operative_schedule` function.
3. It will then reply with a special message containing the *call_id*, the *function name* and the optional *parameters* to use.
4. The returned message is added to the messages history, so that the LLM knows that it has placed this request.
4. The code will then invoke the `operative_schedule` function with the provided arguments and get the returned value.
5. A new message containing: role = `tool`, id=`call_id`, the `**function name** and the **response value** is created and added to the messages history.
6. The LLM is invoked again, without any user message, forcing the LLM to reason using **system_message**, **tools** and the updated list of **messages**
7. Since no more information is needed to complete the task, the LLM will provide the response.

The code below does exactly what described.  
  
Since this is what happens under the hood of the whole Agent story, spend some time playing with it (you can re-run the cell as many times you want), check the content of the LLM response, edit the prompt (e.g. adding **'I am user 1234'**), etc. 

If you are using VS Code, try debugging the cell code using the **Debug Cell** option so you test the whole flow. 
  
<img src="../docs/images/debug-cell.png" width="300">

In [ ]:
# This is the user request, play with it! 
user_message = "are you open tomorrow at 7:30 PM? I am user 1234"

messages = [{'role': 'system', 'content': system_message}]

now = datetime.now().strftime("%A, %d/%m/%Y %H:%M")
input_request = f"Given that now is {now}, handle the following user request enclosed in triple backticks: ```{user_message}```" 
cprint(f"Input: {input_request}", "blue")  

# Pass the request to the LLM
response = invoke_llm(input=input_request)

# Check if the response contains the request to invoke the functions
if response.tool_calls:
    cprint(response, "cyan")
    # Note that we add the request from the LLM to the messages list
    messages.append(response)
    # We iterate over the tool calls to invoke the functions (it is possible to have multiple functions in the same response)
    for tool_call in response.tool_calls:        
        if tool_call.function.name == "operative_schedule":
            tool_args = json.loads(tool_call.function.arguments)  
            cprint (f"Invoking function: {tool_call.function.name}...", "green")
            # We invoke the function and get the response
            function_response=operative_schedule(**tool_args)
            cprint (f"Function call completed...", "yellow")
            # We now add the response from the function to the messages list, note the use of the tool_call.id to let the model link the response to the function call
            messages.append(
                {
                    "tool_call_id": tool_call.id,
                    "role": "tool",
                    "name": tool_call.function.name,
                    "content": function_response,
                })      
                  
    cprint (f"Control back to the LLM...", "magenta")    
    response = invoke_llm()
    cprint (response.content, "yellow")

**WOW** 🥵    

This has been a hard one!  

But now you are familiar with LLM Function Calling and know what makes Agents work, and, contrary to popular belief, know that the LLM is **not** invoking the functions for you. Your code does.

❗**Important**  

The LLM uses the information provided in the `tools` object to understand which tool to invoke, so using a detailed description for both function and parameters is **crucial** to help the LLM understand when (or not) to invoke a particular tool.

Since the `tools` object become part of the prompt, keep in mind that it contributes to the whole request token count, so don't make descriptions extremely long.

In the [Notebook 4](04_get_menu_tool.ipynb) we're going to add another tool for getting the daily menu.